# motif富集

In [ ]:
#-*-coding:utf-8-*-
import json
import numpy as np
import random
from tqdm import tqdm
import math 
import re
import pandas as pd
import pickle
import pickle5
import networkx as nx
import os 
import datetime
from multiprocessing import Pool
from matplotlib import pyplot as plt

In [ ]:
###读入所有位点SER值
os.chdir('/home/dell-none/Desktop/0426jiqun')
pos_value=pd.read_csv('slope_all_SER.csv')
pos_value.head()

###箱图查看整体分布情况
plt.boxplot(list(pos_value['SER']))

###提取10%
plt.boxplot(list(pos_value['SSR']))
min,max=np.percentile(list(pos_value['SSR']),[10,90])
# outlier=np.percentile(list(pos_value['SSR']),75)+1.5*(np.percentile(list(pos_value['SSR']),75)-
                                                  np.percentile(list(pos_value['SSR']),25))
# print(outlier)
print(min)
print(max)
min_df=pos_value[(pos_value["SER"] < min)].reset_index(drop=True)
# max_df=pos_value[(pos_value["SER"] > max)].reset_index(drop=True)

###以低于10%与其他区域比较富集motif

min_df.to_csv('min_df_update.csv',index=False,sep=',')
max_df.to_csv('max_df_update.csv',index=False,sep=',')

###手工检查区域，注意此处应用ncbi位点，并整理到class_region.xlsx文件中

#high:2173:2773
#5122:5680
#13954:14833
#19462:19576
#25963：26155
#26179:26221
#27871：27967
#28084：28333

#low:
#1306:1551
#2773:3679
#4279:4501
#8926:10336
#15061:15199
#16060:16384
#20005:20359
#21289:21340
#22720:22738
#23458：23989


def get_dic(filename):
    dic = {}
    with open(filename, 'r')as f1:
        lines = f1.readlines()
    for line in lines:
        if '>' in line:
            epi = line.rstrip('\n')
            dic[epi] = ''
        else:
            dic[epi] += line.strip('\n')
    return dic 

def write_file(file,dict):
    with open(file,'w')as f2:
        for k1,v1 in dict.items():
            f2.write(k1+'\n'+v1+'\n')


dic1=get_dic('/home/dell-none/Downloads/sqr/sar2_0915_seq/update_net2/seq.fasta')


###提取低于10%的序列片段，及其他片段
re_out=pd.read_excel('class_region.xlsx')
h=list(re_out['Region Positon'])[:8];l=list(re_out['Region Positon'])[8:18]
hgh={};low={};
for k,v in dic1.items():
    for j1 in h:
        key1=k+'_'+j1
        n1,n2=int(j1.split(':')[0]),int(j1.split(':')[1])
        hgh[key1]=v[n1:n2]
        
    for j2 in l:
        key2=k+'_'+j2
        n3,n4=int(j2.split(':')[0]),int(j2.split(':')[1])
#         print(n3,n4)
        
        low[key2]=v[n3:n4]
        
non_h={};non_l={}
##先整理区段
nh=[];nl=[]
for i in range(len(h)):
    if  i ==0:
        nh.append("0:"+h[i].split(':')[0])
    if i==-1:
        nh.append(h[i].split(':')[1]+":29118")
    else:
        nh.append(h[i-1].split(':')[1]+":"+h[i].split(':')[0])
        
        
for i in range(len(l)):
    if  i ==0:
        nl.append("0:"+l[i].split(':')[0])
    if i==-1:
        nl.append(l[i].split(':')[1]+":29118")
    else:
        nl.append(l[i-1].split(':')[1]+":"+l[i].split(':')[0])
        
nhgh={};nlow={};
for k,v in dic1.items():
    for j1 in nh:
        key1=k+'_'+j1
        n1,n2=int(j1.split(':')[0]),int(j1.split(':')[1])
        nhgh[key1]=v[n1:n2]
        
    for j2 in nl:
        key2=k+'_'+j2
        n3,n4=int(j2.split(':')[0]),int(j2.split(':')[1])
#         print(n3,n4)
        
        nlow[key2]=v[n3:n4]
        
###各个序列片段写入fasta文件中       
write_file('hgh_region_v2.fasta',hgh)
write_file('low_region_v2.fasta',low)


write_file('non_hgh_region_v2.fasta',nhgh)
write_file('non_low_region_v2.fasta',nlow)




###meme_suite中back_seq超出可上传大小，对序列去冗余，序列完全一致的随机选择一条。
def get_only(dic):

    re={}

    for v in dic.values():
        if v not in re.keys():
            re[v]=[]
#     print(len(re.keys()))
    ### 获得唯一的序列字典
    for k1 in tqdm(re.keys()):
        for k,v in dic.items():
            if v==k1:
                re[k1].append(k)
#     print(len(re)) 
    return re

low_re=get_only(low);high_re=get_only(hgh)

tran={}
import random
for k,v in low_re.items():
    s=random.sample(v,1)
    tran[s[0]]=k
write_file('low_redun_region_v2.fasta',tran)


tran={}

for k,v in high_re.items():
    s=random.sample(v,1)
    tran[s[0]]=k
write_file('high_redun_region_v2.fasta',tran)


nlow_re=get_only(nlow);nhigh_re=get_only(nhgh)

def writ_re(dic):
    tran={}
    for k,v in dic.items():
        s=random.sample(v,1)
        tran[s[0]]=k
        
    return tran
tran1=writ_re(nlow_re)
tran2=writ_re(nhigh_re)

def del_short(dic):
    new={}
    for k,v in dic.items():
        if len(v)>30:
            new[k]=v
    return new
new1=del_short(tran1)
new2=del_short(tran2)

write_file('nlow_redun_region_v2.fasta',new1)
write_file('nhigh_redun_region_v2.fasta',new2)

####以low_redun_region_v2.fasta为target，以nlow_redun_region_v2.fasta为background，使用meme_suite富集motif

# motif性质

In [ ]:
os.chdir('/home/dell-none/Desktop/0426jiqun/0510high_low')
###使用PSSM矩阵计算motif中各碱基的比例
def cal_acgt(file):
    with open(file,'r')as f1:
            lines=f1.readlines()
    new1={}   
    for i in range(len(lines)):
        if lines[i].startswith('MOTIF'):
            name=lines[i].split(' ')[1]
            k='>num'+name.split('-')[0]+'_'+file.split('.')[0]
            motif=lines[i].split(' ')[1].split('-')[1]
            alst=[i.strip().split(' ') for i in lines[i+2:i+2+len(motif)]]
            a=[list(map(float,i)) for i in alst]
            ####提取PSSM中碱基的百分比,可根据每个位置算出一个平均值
            pssm=pd.DataFrame(a,columns=['A','C','G','T'])
            sum_=np.mean(pssm.iloc[:,0])+np.mean(pssm.iloc[:,1])+np.mean(pssm.iloc[:,2])+np.mean(pssm.iloc[:,3])
            per=[np.mean(pssm.iloc[:,i])/sum_ for i in range(4)]

            new1[k]=per
    return new1

# low=cal_acgt('low.txt')
# high=cal_acgt('high.txt')
low=cal_acgt('low_exceptall_motif.txt')

###计算motif的长度
def motif_length(file):
    with open(file,'r')as f1:
            lines=f1.readlines()

    new={}

    for i in range(len(lines)):
        if lines[i].startswith('MOTIF'):
            name=lines[i].split(' ')[1]
            k='>num'+name.split('-')[0]+'_low'
            motif=lines[i].split(' ')[1].split('-')[1]
            new[k]=len(name)
    return new

low_length=motif_length('low_exceptall_motif.txt')
# high_length=motif_length('high.txt')

a,b=[low],[low_length]
def trans_df(dic,dic1):
    df=pd.DataFrame(dic).T
    df.columns=['A','C','G','T']
    df['length']=dic1.values()
    
    return df


motif_info=pd.concat([trans_df(a[i],b[i]) for i in range(len(a))])
motif_info      

###比例及长度存入文件
motif_info.to_csv('motif_info_fig.csv',index=True,sep=',')

In [ ]:
###长度进行统计学检验
import scipy.stats
leng_compare=[]
for k,v in df.groupby('type'):
    leng_compare.append(list(v['length']))
scipy.stats.ttest_ind(leng_compare[0],leng_compare[1])

###长度间无统计学差异

title=list(df.columns)

###碱基比例统计学检验
import scipy.stats
leng_compare=[];a,t,c,g=[],[],[],[]
for k,v in df.groupby('type'):
    leng_compare.append(list(v['length']))
    
    a.append(list(v['A']))
    t.append(list(v['T']))
    c.append(list(v['C']))
    g.append(list(v['G']))

    
print(scipy.stats.ttest_ind(leng_compare[0],leng_compare[1]))
print(scipy.stats.ttest_ind(a[0],a[1]))
print(scipy.stats.ttest_ind(t[0],t[1]))
print(scipy.stats.ttest_ind(c[0],c[1]))
print(scipy.stats.ttest_ind(g[0],g[1]))

In [ ]:
###以meme_suite 输出的pssm矩阵文件为目标，以参考序列的fasta文件，用FIMO扫描motif所对应的位置得到motif_low_match.tsv文件

In [ ]:
###以meme_suite 输出的pssm矩阵文件为目标，选择meme_suite默认的Ray_rna_motif 数据库为目标，搜寻可结合人类蛋白的相似motif
###得到RBP_match_result.tsv文件